In [3]:
import os
import import_ipynb
import crypto_api as ca

importing Jupyter notebook from crypto_api.ipynb


In [4]:
# List of crypto symbols and IDs
syms = ['btc', 'xrp', 'doge', 'ltc', 'xmr', 'bch', 'xlm', 'bsv', 'zec', 'dash']
cids = ['bitcoin', 'ripple', 'dogecoin', 'litecoin', 'monero', 'bitcoin-cash', 'stellar', 'bitcoin-cash-sv', 'zcash', 'dash']

# Variables for API call
currency = 'usd'
start_date = '2019/01/01'
end_date = '2022/12/31'

In [5]:
# Create dictionary of crypto dataframes
crypto = ca.create_crypto_dict(syms, cids, currency, start_date, end_date)

In [6]:
# Create Excel file with crypto data
file_path = os.path.join("data", "api-data.xlsx")
ca.create_excel(file_path, crypto)